# Finite Difference Operators and Stencils

## Author: Zach Etienne

## Exploring NRPy's finite difference infrastructure, this notebook focuses on `finite_difference.py`: how it constructs finite difference matrices, extracts derivative operators from SymPy expressions, builds stencils and memory access patterns, and generates reusable C helper functions for derivatives. The goal is to learn finite difference features by working through hands on examples, not by reading the source code.

### Required reading if you are unfamiliar with programming or [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system). Otherwise, use for reference; you should be able to pick up the syntax as you follow the tutorial.
* **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
* **[SymPy Tutorial](http://docs.sympy.org/latest/tutorial/intro.html)**

### NRPy Source Code for this module:  
* [finite_difference.py](../edit/finite_difference.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-Python/NRPy-finite-difference-modules): Initialize core Python/NRPy finite difference modules
1. [Step 2](#Step-2:-Explore-finite-difference-matrices-and-stencil-layout): Explore finite difference matrices and stencil layout
1. [Step 3](#Step-3:-Compute-finite-difference-coefficients-and-stencils-from-derivative-strings): Compute finite difference coefficients and stencils from derivative strings
1. [Step 4](#Step-4:-Automatically-discover-derivative-variables-inside-SymPy-expressions): Automatically discover derivative variables inside SymPy expressions
1. [Step 5](#Step-5:-Decode-derivative-variables-into-base-gridfunctions-and-operators): Decode derivative variables into base gridfunctions and operators
1. [Step 6](#Step-6:-Generate-memory-read-code-for-gridfunctions-from-stencils): Generate memory read code for gridfunctions from stencils
1. [Step 7](#Step-7:-Build-prototype-finite-difference-operators-as-SymPy-expressions): Build prototype finite difference operators as SymPy expressions
1. [Step 8](#Step-8:-Turn-prototype-operators-into-reusable-C-finite-difference-functions): Turn prototype operators into reusable C finite difference functions
1. [Step 9](#Step-9:-Upwind-and-Kreiss-Oliger-examples-in-a-small-1D-advection-demo): Upwind and Kreiss Oliger examples in a small 1D advection demo

# Step 1: Initialize core Python/NRPy finite difference modules
### [Back to [top](#Table-of-Contents)]

In this step we:

* Import SymPy and key NRPy modules.
* Import the finite difference helper module.
* Set an infrastructure parameter that will later allow us to enable SIMD friendly code.

From here on, every step will build on these imports, so run this cell once before moving on.

In [1]:
# Step 1: Initialize core Python/NRPy finite difference modules

import sympy as sp

import nrpy.finite_difference as fin   # This notebook's main focus
import nrpy.grid as gri                # Gridfunction registration and metadata
import nrpy.indexedexp as ixp          # Convenient creation of indexed SymPy symbols
import nrpy.params as par              # Parameter interface (Infrastructure, etc.)
import nrpy.c_function as cfc          # Lightweight C function wrapper used by finite_difference

# For SIMD related features, finite_difference expects an Infrastructure value
par.set_parval_from_str("Infrastructure", "BHaH")

# Just to keep the global gridfunction registry predictable while we experiment:
gri.glb_gridfcs_dict.clear()

# Step 2: Explore finite difference matrices and stencil layout
### [Back to [top](#Table-of-Contents)]

NRPy's finite difference module constructs a small matrix $A$ for each stencil type, then inverts it to obtain all derivative stencils of a given order. The helper
`setup_FD_matrix__return_inverse(stencil_width, UPDOWNWIND_stencil_shift)`
returns the inverse of this matrix, which encodes the finite difference coefficients.

* `stencil_width` is the number of grid points in the stencil (for example, 3 for a standard second order centered derivative).
* `UPDOWNWIND_stencil_shift` controls the offset of the stencil relative to the grid point where the derivative is evaluated:
  * `0` gives centered stencils.
  * Positive values shift the stencil upwind.
  * Negative values shift the stencil downwind.

Let us inspect a few matrices for intuition.

In [2]:
# Step 2: Explore the structure of finite difference matrices

# A 3 point centered stencil, e.g. for a second order accurate first derivative
stencil_width_centered = 3
Minv_centered = fin.setup_FD_matrix__return_inverse(
    stencil_width=stencil_width_centered,
    UPDOWNWIND_stencil_shift=0,
)

print("Inverse FD matrix for 3 point centered stencil (shift = 0):")
sp.pprint(Minv_centered)

print("\nEach row of this matrix corresponds to a different derivative order.")
print("Row 0: function value, Row 1: first derivative, Row 2: second derivative.")

Inverse FD matrix for 3 point centered stencil (shift = 0):
⎡0  -1/2  1/2⎤
⎢            ⎥
⎢1   0    -1 ⎥
⎢            ⎥
⎣0  1/2   1/2⎦

Each row of this matrix corresponds to a different derivative order.
Row 0: function value, Row 1: first derivative, Row 2: second derivative.


In [3]:
# A 5 point centered stencil (4th order accurate)
stencil_width_5 = 5
Minv_5_centered = fin.setup_FD_matrix__return_inverse(
    stencil_width=stencil_width_5,
    UPDOWNWIND_stencil_shift=0,
)

print("Inverse FD matrix for 5 point centered stencil (shift = 0):")
sp.pprint(Minv_5_centered)

Inverse FD matrix for 5 point centered stencil (shift = 0):
⎡0  1/12   -1/24  -1/12  1/24⎤
⎢                            ⎥
⎢0  -2/3    2/3    1/6   -1/6⎥
⎢                            ⎥
⎢1    0    -5/4     0    1/4 ⎥
⎢                            ⎥
⎢0   2/3    2/3   -1/6   -1/6⎥
⎢                            ⎥
⎣0  -1/12  -1/24  1/12   1/24⎦


In [4]:
# A 5 point upwinded stencil, shifted by +2 grid points
Minv_5_upwind = fin.setup_FD_matrix__return_inverse(
    stencil_width=stencil_width_5,
    UPDOWNWIND_stencil_shift=2,
)

print("Inverse FD matrix for 5 point upwinded stencil (shift = +2):")
sp.pprint(Minv_5_upwind)

Inverse FD matrix for 5 point upwinded stencil (shift = +2):
⎡   -25    35               ⎤
⎢1  ────   ──    -5/12  1/24⎥
⎢    12    24               ⎥
⎢                           ⎥
⎢0   4    -13/3   3/2   -1/6⎥
⎢                           ⎥
⎢0   -3   19/4    -2    1/4 ⎥
⎢                           ⎥
⎢0  4/3   -7/3    7/6   -1/6⎥
⎢                           ⎥
⎢          11               ⎥
⎢0  -1/4   ──    -1/4   1/24⎥
⎣          24               ⎦


In these examples:

* Column indices encode offsets from the central point.
* Rows encode which derivative (0th, 1st, 2nd, ...) is being approximated.

For example, in the 3 point centered case, the row corresponding to the first derivative at direction 0 is proportional to the familiar second order formula
$$f'(x) \approx \frac{f(x + \Delta x) - f(x - \Delta x)}{2 \Delta x}.$$

We will not usually interact with this matrix directly. Instead, the utility
`compute_fdcoeffs_fdstencl` wraps this in a more user friendly interface that also tracks grid offsets.

# Step 3: Compute finite difference coefficients and stencils from derivative strings
### [Back to [top](#Table-of-Contents)]

The main low level entry point for finite difference coefficients and stencil offsets is

```python
fdcoeffs, fdstencl = fin.compute_fdcoeffs_fdstencl(derivstring, fd_order)
```

where:

* `derivstring` encodes the derivative type and direction(s), using NRPy conventions like:
  * `"dD0"`  for a first derivative in direction 0 (centered unless modified).
  * `"dupD0"` for single point upwind in direction 0.
  * `"ddnD0"` for single point downwind.
  * `"dDD01"` for a mixed second derivative in directions 0 and 1.
  * `"dKOD0"` for Kreiss Oliger dissipation in direction 0.
* `fd_order` is the accuracy order (for example 2 or 4).

The outputs are:

* `fdcoeffs`: list of SymPy `Rational` coefficients.
* `fdstencl`: list of stencil offsets, each an integer triplet `[i0_offset, i1_offset, i2_offset]`.

Let us build and inspect several common derivative types.

In [5]:
# Step 3: Compute finite difference coefficients and stencil offsets

def pretty_print_stencil(name, fdcoeffs, fdstencl):
    print(f"{name}:")
    for coeff, offset in zip(fdcoeffs, fdstencl):
        # Convert SymPy Rational to string before formatting to avoid TypeError
        print(f"  coeff = {str(coeff):>8}, offset = {offset}")
    print("")


# Example 3a: Centered first derivative in x (direction 0), 2nd order
fdcoeffs_dD0_2, fdstencl_dD0_2 = fin.compute_fdcoeffs_fdstencl("dD0", fd_order=2)
pretty_print_stencil("dD0, order 2", fdcoeffs_dD0_2, fdstencl_dD0_2)

# Example 3b: Centered first derivative in x, 4th order
fdcoeffs_dD0_4, fdstencl_dD0_4 = fin.compute_fdcoeffs_fdstencl("dD0", fd_order=4)
pretty_print_stencil("dD0, order 4", fdcoeffs_dD0_4, fdstencl_dD0_4)

# Example 3c: Mixed second derivative d^2/(dx dy), 2nd order
fdcoeffs_dDD01_2, fdstencl_dDD01_2 = fin.compute_fdcoeffs_fdstencl("dDD01", fd_order=2)
pretty_print_stencil("dDD01, order 2", fdcoeffs_dDD01_2, fdstencl_dDD01_2)

# Example 3d: Single point upwind derivative in x (direction 0)
fdcoeffs_dupD0_2, fdstencl_dupD0_2 = fin.compute_fdcoeffs_fdstencl("dupD0", fd_order=2)
pretty_print_stencil("dupD0, order 2", fdcoeffs_dupD0_2, fdstencl_dupD0_2)

# Example 3e: Kreiss Oliger dissipation in y (direction 1), order 4
fdcoeffs_dKOD1_4, fdstencl_dKOD1_4 = fin.compute_fdcoeffs_fdstencl("dKOD1", fd_order=4)
pretty_print_stencil("dKOD1, KO, base order 4", fdcoeffs_dKOD1_4, fdstencl_dKOD1_4)

dD0, order 2:
  coeff =     -1/2, offset = [-1, 0, 0]
  coeff =      1/2, offset = [1, 0, 0]

dD0, order 4:
  coeff =     1/12, offset = [-2, 0, 0]
  coeff =     -2/3, offset = [-1, 0, 0]
  coeff =      2/3, offset = [1, 0, 0]
  coeff =    -1/12, offset = [2, 0, 0]

dDD01, order 2:
  coeff =      1/4, offset = [-1, -1, 0]
  coeff =     -1/4, offset = [1, -1, 0]
  coeff =     -1/4, offset = [-1, 1, 0]
  coeff =      1/4, offset = [1, 1, 0]

dupD0, order 2:
  coeff =     -3/2, offset = [0, 0, 0]
  coeff =        2, offset = [1, 0, 0]
  coeff =     -1/2, offset = [2, 0, 0]

dKOD1, KO, base order 4:
  coeff =     1/64, offset = [0, -3, 0]
  coeff =    -3/32, offset = [0, -2, 0]
  coeff =    15/64, offset = [0, -1, 0]
  coeff =    -5/16, offset = [0, 0, 0]
  coeff =    15/64, offset = [0, 1, 0]
  coeff =    -3/32, offset = [0, 2, 0]
  coeff =     1/64, offset = [0, 3, 0]



Reading these outputs:

* Each `offset` tells you how far from the central point the grid value is, in units of grid spacing.
* For example, the 2nd order centered first derivative in direction 0 uses offsets `[ -1, 0, 0 ]` and `[ 1, 0, 0 ]` with symmetric coefficients, as expected.
* Mixed second derivatives combine two one dimensional stencils and produce offsets where more than one entry in the triplet is nonzero.

In practice, you rarely have to construct `derivstring` by hand. Higher level helpers can extract them automatically from SymPy expressions, as we will see next.

# Step 4: Automatically discover derivative variables inside SymPy expressions
### [Back to [top](#Table-of-Contents)]

The function

```python
fin.extract_list_of_deriv_var_strings_from_sympyexpr_list(
    list_of_free_symbols,
    upwind_control_vec,
)
```

scans a collection of SymPy expressions and returns a list of symbols that look like finite difference derivatives. The function knows the naming patterns used by NRPy:

* Suffixes like `_dD`, `_dupD`, `_ddnD`, and `_dKOD`.
* Components indicated by trailing digits (for example `u_dD0`).

It distinguishes these from:

* Registered gridfunctions.
* Registered C parameters.
* Generic symbols that do not match derivative naming rules.

As a warm up, let us classify a few symbols, then let the helper search for derivatives inside an expression.

In [6]:
# Step 4: Discover derivative variables inside SymPy expressions

# First register a simple scalar gridfunction and a vector gridfunction
gri.glb_gridfcs_dict.clear()
u_gf = gri.register_gridfunctions("u")[0]                 # scalar gridfunction "u"
vecU = gri.register_gridfunctions_for_single_rank1("vecU")  # components vecU[0], vecU[1], vecU[2]

# Register some code parameters
alpha, beta = par.register_CodeParameters(
    cparam_type="double",
    module="fd_demo",
    names=["alpha", "beta"],
    defaultvalues=[1.0, 2.0],
)

# Basic classification examples
print("Classification examples:")
for symbol in [u_gf, vecU[0], alpha, sp.Symbol("plain_symbol")]:
    kind = fin.symbol_is_gridfunction_Cparameter_or_other(symbol)
    print(f"  {symbol}: {kind}")
print("")

# Now declare derivative symbols using indexedexp
u_dD = ixp.declarerank1("u_dD")       # u_dD[0], u_dD[1], u_dD[2]
u_dupD = ixp.declarerank1("u_dupD")   # upwind derivative symbols

# Build a small SymPy expression that mixes everything
expr = alpha * u_dD[0] * u_gf + beta * u_dupD[0] + vecU[1]

print("Expression:")
print("  ", expr)

# Collect free symbols from this expression
free_symbols_list = list(expr.free_symbols)

print("\nFree symbols in the expression:")
for symb in free_symbols_list:
    print("  ", symb)

# Ask finite_difference to find derivative variables, without and with upwind control
derivs_no_upwind = fin.extract_list_of_deriv_var_strings_from_sympyexpr_list(
    free_symbols_list,
    upwind_control_vec="unset",   # string means "do not add matching ddn derivatives automatically"
)

derivs_with_upwind = fin.extract_list_of_deriv_var_strings_from_sympyexpr_list(
    free_symbols_list,
    upwind_control_vec=sp.Symbol("beta_control"),  # any non string means "enable upwind logic"
)

print("\nDerivative variables found without explicit upwind control:")
print("  ", derivs_no_upwind)

print("\nDerivative variables found with upwind control enabled:")
print("  ", derivs_with_upwind)

Classification examples:
  u: gridfunction
  vecU0: gridfunction
  alpha: Cparameter
  plain_symbol: other

Expression:
   alpha*u*u_dD0 + beta*u_dupD0 + vecU1

Free symbols in the expression:
   beta
   u_dupD0
   vecU1
   u
   u_dD0
   alpha

Derivative variables found without explicit upwind control:
   [u_dD0, u_dupD0]

Derivative variables found with upwind control enabled:
   [u_dD0, u_ddnD0, u_dupD0]


Things to notice:

* `symbol_is_gridfunction_Cparameter_or_other` distinguishes gridfunctions, C parameters, and generic symbols.
* `extract_list_of_deriv_var_strings_from_sympyexpr_list` only flags symbols whose names match finite difference derivative patterns.
* When `upwind_control_vec` is not a string, any symbol with `_dupD` in its name causes a matching `_ddnD` symbol to be added automatically. This makes it easy to build schemes that depend on an upwind control vector without having to list both directions by hand.

# Step 5: Decode derivative variables into base gridfunctions and operators
### [Back to [top](#Table-of-Contents)]

Derivative variables such as `u_dD0` or `hDD_dDD12` encode two separate pieces of information:

* The base gridfunction, for example `u` or `hDD01`.
* The derivative operator and its directions, for example `dD0` or `dDD12`.

The helper

```python
base_gf_names, deriv_ops = fin.extract_base_gfs_and_deriv_ops_lists__from_list_of_deriv_vars(
    list_of_deriv_vars,
)
```

splits these symbols into:

* A list of base gridfunction name strings.
* A parallel list of derivative operator strings.

This is exactly what we want to feed into `compute_fdcoeffs_fdstencl` and later into stencil based code generators.

In [7]:
# Step 5: Decode derivative variables into base gridfunctions and operators

# Reuse the derivative list from Step 4
list_of_deriv_vars = derivs_with_upwind

base_gf_names, deriv_ops = fin.extract_base_gfs_and_deriv_ops_lists__from_list_of_deriv_vars(
    list_of_deriv_vars
)

print("Decoded derivatives:")
for var, gf_name, op in zip(list_of_deriv_vars, base_gf_names, deriv_ops):
    # Convert SymPy Symbols to strings before applying alignment
    print(f"  symbol = {str(var):>10}  ->  base gridfunction = {str(gf_name):>6}, operator = {op}")

Decoded derivatives:
  symbol =      u_dD0  ->  base gridfunction =      u, operator = dD0
  symbol =    u_ddnD0  ->  base gridfunction =      u, operator = ddnD0
  symbol =    u_dupD0  ->  base gridfunction =      u, operator = dupD0


At this point we know:

* Which gridfunctions need derivatives.
* Which derivative operator string applies to each.

In the next step we will combine this information with the finite difference coefficient helper to generate efficient memory read code.

# Step 6: Generate memory read code for gridfunctions from stencils
### [Back to [top](#Table-of-Contents)]

Once we know:

* The base gridfunction name for each derivative.
* The finite difference stencil offsets for each derivative.
* The list of free symbols that appear in our expressions.

we can ask `finite_difference.py` to build optimized C code that:

* Reads exactly the grid points needed for all derivatives.
* Avoids redundant loads (no reading the same point twice).
* Orders memory accesses to reduce cache misses.
* Optionally uses SIMD friendly loads when enabled.

The key helper is

```python
read_code = fin.read_gfs_from_memory(
    list_of_base_gridfunction_names_in_derivs,
    fdstencl,
    free_symbols_list,
    mem_alloc_style,
    enable_simd,
)
```

Here we demonstrate:

* Generating the stencils for our derivative operators.
* Calling `read_gfs_from_memory` with a simple memory allocation style.
* Comparing scalar and SIMD memory read code.

In [8]:
# Step 6: Generate memory read code from stencils

# Build FD stencils corresponding to the derivative operators we decoded in Step 5
fdcoeffs_all = []
fdstencl_all = []

fd_order_demo = 2

for op in deriv_ops:
    coeffs, stencils = fin.compute_fdcoeffs_fdstencl(op, fd_order=fd_order_demo)
    fdcoeffs_all.append(coeffs)
    fdstencl_all.append(stencils)

print("Stencil data for each derivative operator:")
for op, coeffs, stencils in zip(deriv_ops, fdcoeffs_all, fdstencl_all):
    print(f"  Operator {op}:")
    for coeff, offset in zip(coeffs, stencils):
        print(f"    coeff = {str(coeff):>6}, offset = {offset}")
    print("")

# Use the free symbols collected in Step 4 (free_symbols_list)
print("Generating scalar memory read code (no SIMD, mem_alloc_style = '210'):\n")

read_code_scalar = fin.read_gfs_from_memory(
    list_of_base_gridfunction_names_in_derivs=base_gf_names,
    fdstencl=fdstencl_all,
    free_symbols_list=free_symbols_list,
    mem_alloc_style="210",
    enable_simd=False,
)

print(read_code_scalar)

Stencil data for each derivative operator:
  Operator dD0:
    coeff =   -1/2, offset = [-1, 0, 0]
    coeff =    1/2, offset = [1, 0, 0]

  Operator ddnD0:
    coeff =    1/2, offset = [-2, 0, 0]
    coeff =     -2, offset = [-1, 0, 0]
    coeff =    3/2, offset = [0, 0, 0]

  Operator dupD0:
    coeff =   -3/2, offset = [0, 0, 0]
    coeff =      2, offset = [1, 0, 0]
    coeff =   -1/2, offset = [2, 0, 0]

Generating scalar memory read code (no SIMD, mem_alloc_style = '210'):

const REAL u_i0m2 = in_gfs[IDX4(UGF, i0-2, i1, i2)];
const REAL u_i0m1 = in_gfs[IDX4(UGF, i0-1, i1, i2)];
const REAL u = in_gfs[IDX4(UGF, i0, i1, i2)];
const REAL u_i0p1 = in_gfs[IDX4(UGF, i0+1, i1, i2)];
const REAL u_i0p2 = in_gfs[IDX4(UGF, i0+2, i1, i2)];
const REAL vecU1 = in_gfs[IDX4(VECU1GF, i0, i1, i2)];



In [9]:
# Now enable SIMD and switch to the "012" memory layout to see how the code changes

print("Generating SIMD memory read code (enable_simd = True, mem_alloc_style = '012'):\n")

read_code_simd = fin.read_gfs_from_memory(
    list_of_base_gridfunction_names_in_derivs=base_gf_names,
    fdstencl=fdstencl_all,
    free_symbols_list=free_symbols_list,
    mem_alloc_style="012",
    enable_simd=True,
)

print(read_code_simd)

Generating SIMD memory read code (enable_simd = True, mem_alloc_style = '012'):

const REAL_SIMD_ARRAY u_i0m2 = ReadSIMD(&in_gfs[IDX4(UGF, i0-2, i1, i2)]);
const REAL_SIMD_ARRAY u_i0m1 = ReadSIMD(&in_gfs[IDX4(UGF, i0-1, i1, i2)]);
const REAL_SIMD_ARRAY u = ReadSIMD(&in_gfs[IDX4(UGF, i0, i1, i2)]);
const REAL_SIMD_ARRAY u_i0p1 = ReadSIMD(&in_gfs[IDX4(UGF, i0+1, i1, i2)]);
const REAL_SIMD_ARRAY u_i0p2 = ReadSIMD(&in_gfs[IDX4(UGF, i0+2, i1, i2)]);
const REAL_SIMD_ARRAY vecU1 = ReadSIMD(&in_gfs[IDX4(VECU1GF, i0, i1, i2)]);



Take a look at the output:

* For each gridfunction and each unique offset, exactly one `const` is defined.
* The variable names are built from a base name plus suffixes like `_i0m1_i2p2`, which encode offsets in each direction.
* When `enable_simd=False`, the type is the standard floating point alias (for example `REAL`).
* When `enable_simd=True`, the type becomes `REAL_SIMD_ARRAY` and the loads are wrapped in SIMD helper macros such as `ReadSIMD`.

You do not have to hand write any of these memory access patterns; the finite difference helper keeps them consistent and cache friendly.

# Step 7: Build prototype finite difference operators as SymPy expressions
### [Back to [top](#Table-of-Contents)]

The finite difference module can build *prototype* derivative operators that act on a symbolic gridfunction name such as `"FDPROTO"`. This is very useful for:

* Inspecting the discrete operator in symbolic form.
* Reusing the same stencil for many different gridfunctions.
* Generating reusable C helper functions for multiple codes.

The main entry point is

```python
FDexprs, FDlhsvarnames, symbol_to_Rational = fin.proto_FD_operators_to_sympy_expressions(
    list_of_proto_deriv_symbs,
    fd_order,
    fdcoeffs,
    fdstencl,
    enable_simd=False,
)
```

where:

* `list_of_proto_deriv_symbs` is a list of SymPy symbols like `FDPROTO_dD0` or `FDPROTO_dKOD1`.
* `fdcoeffs` and `fdstencl` are the coefficient and stencil lists that match those operators.
* `FDexprs` is a list of SymPy expressions that implement each operator.
* `FDlhsvarnames` is a list of strings for left hand side declarations, for example `const REAL FDPROTO_dD0`.
* `symbol_to_Rational` maps helper symbols to `Rational` values introduced during preprocessing and factorization.

Let us build prototype first derivative and upwind operators and inspect the resulting expressions.

In [10]:
# Step 7: Build prototype FD operators for a symbolic gridfunction

# Clear and register a generic prototype gridfunction
gri.glb_gridfcs_dict.clear()
FDPROTO_gf = gri.register_gridfunctions("FDPROTO")[0]

# Declare prototype derivative symbols for FDPROTO
FDPROTO_dD = ixp.declarerank1("FDPROTO_dD")       # FDPROTO_dD[0], ...
FDPROTO_dupD = ixp.declarerank1("FDPROTO_dupD")   # FDPROTO_dupD[0], ...

# Choose which prototype operators we want to build, and their derivative strings
list_of_proto_deriv_symbs = [FDPROTO_dD[0], FDPROTO_dupD[0]]
list_of_proto_deriv_ops = [str(symb).split("_")[1] for symb in list_of_proto_deriv_symbs]

print("Prototype derivative operator strings:")
for symb, op in zip(list_of_proto_deriv_symbs, list_of_proto_deriv_ops):
    print(f"  {symb}  ->  {op}")

# Build coefficient and stencil lists for each operator
fd_order_proto = 4

fdcoeffs_proto = []
fdstencl_proto = []

for op in list_of_proto_deriv_ops:
    coeffs, stencils = fin.compute_fdcoeffs_fdstencl(op, fd_order=fd_order_proto)
    fdcoeffs_proto.append(coeffs)
    fdstencl_proto.append(stencils)

# Convert these into SymPy expressions
FDexprs, FDlhsvarnames, symbol_to_Rational = fin.proto_FD_operators_to_sympy_expressions(
    list_of_proto_deriv_symbs=list_of_proto_deriv_symbs,
    fd_order=fd_order_proto,
    fdcoeffs=fdcoeffs_proto,
    fdstencl=fdstencl_proto,
    enable_simd=False,
)

print("\nPrototype finite difference expressions:")
for lhs, expr in zip(FDlhsvarnames, FDexprs):
    print(f"{lhs} = {expr}\n")

print("Symbol to Rational mapping used in the expressions:")
for symb, val in symbol_to_Rational.items():
    print(f"  {symb} = {val}")

Prototype derivative operator strings:
  FDPROTO_dD0  ->  dD0
  FDPROTO_dupD0  ->  dupD0

Prototype finite difference expressions:
const REAL FDPROTO_dD0 = invdxx0*(FDPart1_Rational_1_12*(FDPROTO_i0m2 - FDPROTO_i0p2) + FDPart1_Rational_2_3*(-FDPROTO_i0m1 + FDPROTO_i0p1))

const REAL UpwindAlgInputFDPROTO_dupD0 = invdxx0*(-FDPROTO*FDPart1_Rational_5_6 - FDPROTO_i0m1*FDPart1_Rational_1_4 + FDPROTO_i0p1*FDPart1_Rational_3_2 - FDPROTO_i0p2*FDPart1_Rational_1_2 + FDPROTO_i0p3*FDPart1_Rational_1_12)

Symbol to Rational mapping used in the expressions:
  FDPart1_Rational_2_3 = 2/3
  FDPart1_Rational_1_12 = 1/12
  FDPart1_Rational_5_6 = 5/6
  FDPart1_Rational_1_2 = 1/2
  FDPart1_Rational_1_4 = 1/4
  FDPart1_Rational_3_2 = 3/2


In [11]:
# If we enable SIMD, the left hand side declarations change type,
# but the symbolic structure of the operator is unchanged.

FDexprs_simd, FDlhsvarnames_simd, symbol_to_Rational_simd = fin.proto_FD_operators_to_sympy_expressions(
    list_of_proto_deriv_symbs=list_of_proto_deriv_symbs,
    fd_order=fd_order_proto,
    fdcoeffs=fdcoeffs_proto,
    fdstencl=fdstencl_proto,
    enable_simd=True,
)

print("\nLeft hand side declarations with SIMD enabled:")
for lhs in FDlhsvarnames_simd:
    print(" ", lhs)


Left hand side declarations with SIMD enabled:
  const REAL_SIMD_ARRAY FDPROTO_dD0
  const REAL_SIMD_ARRAY UpwindAlgInputFDPROTO_dupD0


You can treat the resulting `FDexprs` like any other SymPy expression:

* Substitute numerical values for `FDPROTO` and the inverse grid spacings `invdxx0`, `invdxx1`, etc.
* Factor, expand, or simplify them further if needed.
* Use them as building blocks when constructing complex PDE right hand sides.

# Step 8: Turn prototype operators into reusable C finite difference functions
### [Back to [top](#Table-of-Contents)]

When `proto_FD_operators_to_sympy_expressions` is called, it also populates an internal dictionary `fin.FDFunctions_dict` with `FDFunction` objects, one for each derivative operator.

Each `FDFunction` knows how to:

* Wrap the finite difference expression into a small C function with a well defined name.
* Provide the function signature and parameter list.
* Generate a C line that calls this helper and assigns its result to a derivative variable.

The finite difference module includes a helper

```python
fin.construct_FD_functions_prefunc()
```

that concatenates the full C definitions of all registered finite difference helper functions.

Before we can call this helper, we must first attach a `cfc.CFunction` object to each `FDFunction`, using its `CFunction_fd_function` method. This small utility does that conversion, using SymPy's C code printer:

In [12]:
# Step 8: Generate reusable C finite difference functions

def attach_CFunctions_to_FDFunctions():
    """
    For each FDFunction in fin.FDFunctions_dict, build a small CFunction
    that evaluates the corresponding FDexpr and returns it. This prepares
    the dictionary for fin.construct_FD_functions_prefunc().
    """
    for fd_func in fin.FDFunctions_dict.values():
        # Build a C assignment FD_result = <expression>;
        assignment = sp.ccode(fd_func.FDexpr, assign_to="FD_result")
        # Declare FD_result with the correct floating point type
        FDexpr_c_code = f"{fd_func.fp_type_alias} FD_result;\n{assignment}"
        # Attach the CFunction object
        fd_func.CFunction = fd_func.CFunction_fd_function(FDexpr_c_code)

# Attach CFunctions for prototype operators defined in Step 7
attach_CFunctions_to_FDFunctions()

# 8a. Generate all FD helper functions that have been recorded so far
prefunc_code = fin.construct_FD_functions_prefunc()

print("Finite difference helper functions generated from prototype operators:\n")
print(prefunc_code)

Finite difference helper functions generated from prototype operators:

/**
 * Finite difference function for operator dD0, with FD accuracy order 4.
 */
static NO_INLINE REAL_SIMD_ARRAY SIMD_fd_function_dD0_fdorder4(const REAL_SIMD_ARRAY FDPROTO_i0m1, const REAL_SIMD_ARRAY FDPROTO_i0m2,
                                                               const REAL_SIMD_ARRAY FDPROTO_i0p1, const REAL_SIMD_ARRAY FDPROTO_i0p2,
                                                               const REAL_SIMD_ARRAY invdxx0) {
  REAL_SIMD_ARRAY FD_result;
  FD_result = invdxx0 * (FDPart1_Rational_1_12 * (FDPROTO_i0m2 + FDPROTO_i0p2 * FDPart1_NegativeOne_) +
                         FDPart1_Rational_2_3 * (FDPROTO_i0m1 * FDPart1_NegativeOne_ + FDPROTO_i0p1));
  return FD_result;
} // END FUNCTION SIMD_fd_function_dD0_fdorder4
/**
 * Finite difference function for operator dupD0, with FD accuracy order 4.
 */
static NO_INLINE REAL_SIMD_ARRAY SIMD_fd_function_dupD0_fdorder4(const REAL_SIMD_ARRAY FDPRO

In [13]:
# 8b. Build an explicit C call to a specific helper function

# Pick the first operator we defined in Step 7, for example "dD0"
example_operator_key = list(fin.FDFunctions_dict.keys())[0]
example_fd_function = fin.FDFunctions_dict[example_operator_key]

print(f"Using FDFunction stored under key: {example_operator_key}")
print(f"C function name: {example_fd_function.c_function_name}")

# Suppose we want the derivative of a gridfunction named "u" to be stored in "u_dD0"
c_call_line = example_fd_function.c_function_call(
    gf_name="u",
    deriv_var="u_dD0",
)

print("\nExample C call line:")
print(c_call_line + ";")

Using FDFunction stored under key: dD0
C function name: SIMD_fd_function_dD0_fdorder4

Example C call line:
const REAL_SIMD_ARRAY u_dD0 = SIMD_fd_function_dD0_fdorder4(u_i0m1,u_i0m2,u_i0p1,u_i0p2,invdxx0);


At this point you have:

* Short, reusable C functions that encapsulate the finite difference stencil for each operator.
* A convenient way to emit call sites for these helper functions.

Typical usage in a full PDE code looks like:

1. Include the generated helper functions in a "prefunc" section.
2. In your main update loop, call these helpers with the appropriate arguments for each gridfunction.

The advantage is that if you later change the finite difference order or stencil type, you only regenerate these helpers instead of editing the physics code by hand.

# Step 9: Upwind and Kreiss Oliger examples in a small 1D advection demo
### [Back to [top](#Table-of-Contents)]

As a final example, let us sketch how the finite difference helpers can be combined to build a semi discrete 1D advection scheme of the form

$$\partial_t u = - v \partial_x u + \epsilon_{\text{KO}} \cdot \text{KO}(u),$$

where:

* $u$ is a scalar gridfunction.
* $v$ is a constant advection speed.
* `KO(u)` represents a Kreiss Oliger dissipation operator.

We will:

* Encode the right hand side symbolically using upwind and Kreiss Oliger derivative variables.
* Let `finite_difference.py` discover which derivatives are needed.
* Build the corresponding finite difference operators and helper functions.

This is not a full time integration code; the goal is to show the pieces in one place.

In [14]:
# Step 9: Upwind and Kreiss Oliger example for 1D advection

# Clear and register the scalar gridfunction u again
gri.glb_gridfcs_dict.clear()
u = gri.register_gridfunctions("u")[0]

# Declare 1D derivative symbols for u:
#   u_dupD0 : upwind first derivative in x
#   u_dKOD0 : Kreiss Oliger operator in x
u_dupD = ixp.declarerank1("u_dupD")
u_dKOD = ixp.declarerank1("u_dKOD")

# Advection speed and dissipation strength as C parameters
v, eps_KO = par.register_CodeParameters(
    cparam_type="double",
    module="advect_demo",
    names=["v", "eps_KO"],
    defaultvalues=[1.0, 0.1],
)

# Symbolic semi discrete right hand side:
#   rhs = -v * u_x_upwind + eps_KO * KO(u)
rhs_expr = -v * u_dupD[0] + eps_KO * u_dKOD[0]

print("Semi discrete RHS expression:")
print("  rhs =", rhs_expr)

Semi discrete RHS expression:
  rhs = eps_KO*u_dKOD0 - u_dupD0*v


In [15]:
# 9a. Let finite_difference.py find and decode the derivative variables

rhs_free_symbols = list(rhs_expr.free_symbols)

deriv_vars = fin.extract_list_of_deriv_var_strings_from_sympyexpr_list(
    rhs_free_symbols,
    upwind_control_vec=sp.Symbol("v_control"),
)

print("\nDerivative variables appearing in the RHS:")
print("  ", deriv_vars)

base_gf_names_rhs, deriv_ops_rhs = fin.extract_base_gfs_and_deriv_ops_lists__from_list_of_deriv_vars(
    deriv_vars
)

print("\nDecoded derivative information for the RHS:")
for var, base_name, op in zip(deriv_vars, base_gf_names_rhs, deriv_ops_rhs):
    print(f"  symbol = {str(var):>10}  ->  base = {str(base_name):>2}, operator = {op}")


Derivative variables appearing in the RHS:
   [u_dKOD0, u_ddnD0, u_dupD0]

Decoded derivative information for the RHS:
  symbol =    u_dKOD0  ->  base =  u, operator = dKOD0
  symbol =    u_ddnD0  ->  base =  u, operator = ddnD0
  symbol =    u_dupD0  ->  base =  u, operator = dupD0


In [16]:
# 9b. Build finite difference operators for these derivatives, using a specific order

fd_order_advect = 4

fdcoeffs_rhs = []
fdstencl_rhs = []

for op in deriv_ops_rhs:
    coeffs, stencils = fin.compute_fdcoeffs_fdstencl(op, fd_order=fd_order_advect)
    fdcoeffs_rhs.append(coeffs)
    fdstencl_rhs.append(stencils)

print("\nStencil data for RHS operators (order = 4):")
for op, coeffs, stencils in zip(deriv_ops_rhs, fdcoeffs_rhs, fdstencl_rhs):
    print(f"  Operator {op}:")
    for coeff, offset in zip(coeffs, stencils):
        print(f"    coeff = {str(coeff):>6}, offset = {offset}")
    print("")


Stencil data for RHS operators (order = 4):
  Operator dKOD0:
    coeff =   1/64, offset = [-3, 0, 0]
    coeff =  -3/32, offset = [-2, 0, 0]
    coeff =  15/64, offset = [-1, 0, 0]
    coeff =  -5/16, offset = [0, 0, 0]
    coeff =  15/64, offset = [1, 0, 0]
    coeff =  -3/32, offset = [2, 0, 0]
    coeff =   1/64, offset = [3, 0, 0]

  Operator ddnD0:
    coeff =  -1/12, offset = [-3, 0, 0]
    coeff =    1/2, offset = [-2, 0, 0]
    coeff =   -3/2, offset = [-1, 0, 0]
    coeff =    5/6, offset = [0, 0, 0]
    coeff =    1/4, offset = [1, 0, 0]

  Operator dupD0:
    coeff =   -1/4, offset = [-1, 0, 0]
    coeff =   -5/6, offset = [0, 0, 0]
    coeff =    3/2, offset = [1, 0, 0]
    coeff =   -1/2, offset = [2, 0, 0]
    coeff =   1/12, offset = [3, 0, 0]



In [17]:
# 9c. Build prototype versions of these operators and generate helper functions.
# We follow the same naming pattern as Step 7: each prototype symbol name contains
# exactly one underscore, so that split('_')[1] recovers the derivative operator.

# Clear the prototype gridfunction registry and register a generic one
gri.glb_gridfcs_dict.clear()
FDPROTO_adv = gri.register_gridfunctions("FDPROTO")[0]

# IMPORTANT: clear any existing FDFunctions so that the internal bookkeeping in
# proto_FD_operators_to_sympy_expressions sees only the operators for this step.
fin.FDFunctions_dict.clear()

# Create matching prototype symbols FDPROTO_<operator> for each RHS operator
proto_deriv_symbols_adv = []
for op in deriv_ops_rhs:
    proto_name = f"FDPROTO_{op}"
    proto_sym = sp.Symbol(proto_name)
    proto_deriv_symbols_adv.append(proto_sym)

# proto_FD_operators_to_sympy_expressions expects fdcoeffs and fdstencl parallel to these operators
FDexprs_adv, FDlhs_adv, symbol_to_Rational_adv = fin.proto_FD_operators_to_sympy_expressions(
    list_of_proto_deriv_symbs=proto_deriv_symbols_adv,
    fd_order=fd_order_advect,
    fdcoeffs=fdcoeffs_rhs,
    fdstencl=fdstencl_rhs,
    enable_simd=False,
)

print("Prototype FD operators for advection example:")
for lhs, expr in zip(FDlhs_adv, FDexprs_adv):
    print(f"{lhs} = {expr}\n")

Prototype FD operators for advection example:
const REAL FDPROTO_dKOD0 = invdxx0*(-FDPROTO*FDPart1_Rational_5_16 + FDPart1_Rational_15_64*(FDPROTO_i0m1 + FDPROTO_i0p1) + FDPart1_Rational_1_64*(FDPROTO_i0m3 + FDPROTO_i0p3) + FDPart1_Rational_3_32*(-FDPROTO_i0m2 - FDPROTO_i0p2))

const REAL UpwindAlgInputFDPROTO_ddnD0 = invdxx0*(FDPROTO*FDPart1_Rational_5_6 - FDPROTO_i0m1*FDPart1_Rational_3_2 + FDPROTO_i0m2*FDPart1_Rational_1_2 - FDPROTO_i0m3*FDPart1_Rational_1_12 + FDPROTO_i0p1*FDPart1_Rational_1_4)

const REAL UpwindAlgInputFDPROTO_dupD0 = invdxx0*(-FDPROTO*FDPart1_Rational_5_6 - FDPROTO_i0m1*FDPart1_Rational_1_4 + FDPROTO_i0p1*FDPart1_Rational_3_2 - FDPROTO_i0p2*FDPart1_Rational_1_2 + FDPROTO_i0p3*FDPart1_Rational_1_12)



In [18]:
# 9d. Generate helper functions and example call lines for the advection RHS

# Attach CFunctions for the new set of FDFunctions created in Step 9c
attach_CFunctions_to_FDFunctions()

prefunc_adv = fin.construct_FD_functions_prefunc()

print("Helper functions for advection operators:\n")
print(prefunc_adv)

print("Example call lines:")
for key, func in fin.FDFunctions_dict.items():
    call_line = func.c_function_call(
        gf_name="u",           # base gridfunction name in a realistic code
        deriv_var=f"u_{key}",  # where to store the result
    )
    print(" ", call_line + ";")

Helper functions for advection operators:

/**
 * Finite difference function for operator dKOD0, with FD accuracy order 4.
 */
static NO_INLINE REAL fd_function_dKOD0_fdorder4(const REAL FDPROTO, const REAL FDPROTO_i0m1, const REAL FDPROTO_i0m2, const REAL FDPROTO_i0m3,
                                                 const REAL FDPROTO_i0p1, const REAL FDPROTO_i0p2, const REAL FDPROTO_i0p3, const REAL invdxx0) {
  REAL FD_result;
  FD_result = invdxx0 * (-FDPROTO * FDPart1_Rational_5_16 + FDPart1_Rational_15_64 * (FDPROTO_i0m1 + FDPROTO_i0p1) +
                         FDPart1_Rational_1_64 * (FDPROTO_i0m3 + FDPROTO_i0p3) + FDPart1_Rational_3_32 * (-FDPROTO_i0m2 - FDPROTO_i0p2));
  return FD_result;
} // END FUNCTION fd_function_dKOD0_fdorder4
/**
 * Finite difference function for operator ddnD0, with FD accuracy order 4.
 */
static NO_INLINE REAL fd_function_ddnD0_fdorder4(const REAL FDPROTO, const REAL FDPROTO_i0m1, const REAL FDPROTO_i0m2, const REAL FDPROTO_i0m3,
                 

This final step showed how the pieces fit together:

* A high level symbolic RHS using derivative variables (`u_dupD0`, `u_dKOD0`).
* Automated discovery of derivative operators and base gridfunctions.
* Construction of stencils and prototype operators for a chosen order.
* Generation of reusable C helper functions and call sites.

From here, you can:

* Swap in different finite difference orders by changing `fd_order_advect`.
* Switch between centered and upwind derivatives by changing the operator strings.
* Enable SIMD in the prototype helper construction to produce vector friendly operators.

All without editing a single finite difference weight by hand.